In [5]:
from pqdm.processes import pqdm
import matplotlib.pyplot as plt 
from natsort import natsorted 
import os 
import numpy as np 
import pandas as pd 

run = "bigrun15"
for run in ["bigrun15", "bigrun16"]:
  folders   = natsorted([a for a in os.listdir("data/generated/%s/"%run) if ".txt" not in a and "GDB" in a])
  folders   = [a for a in folders if os.path.exists("data/generated/%s/%s/data.csv"%(run, a))]
  filenames = ["data/generated/%s/%s/data.csv"%(run, folders[i]) for i in range(len(folders))] 
  sizes     = [os.path.getsize("data/generated/%s/%s/data.csv"%(run, f)) for f in folders]
  indxs = np.argsort(sizes)
  filenames = np.array(filenames)[indxs].tolist()


  def square(filename):
      return pd.read_csv(filename, compression="gzip").shape[0] # make script that dumpts into another file. 

  # obs: running multiple times it is slower, e.g., queuing takes 2min. 
  result_ = pqdm(filenames, square, n_jobs=100)
  print(len(result_))
  print(np.sum([a for a in result_ if type(a) == type(1)]))

QUEUEING TASKS | :   0%|          | 0/153 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/153 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/153 [00:00<?, ?it/s]

248682276


QUEUEING TASKS | :   0%|          | 0/217 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/217 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/217 [00:00<?, ?it/s]

192512257


In [1]:
from pqdm.processes import pqdm
import matplotlib.pyplot as plt 
from natsort import natsorted 
import os 
import numpy as np 
import pandas as pd 
from periodictable import elements as pt

def square(filename):
    #df = pd.read_csv(filename, compression="gzip", nrows=10000) # for processing everything in 10s to check for bugs 
    df = pd.read_csv(filename, compression="gzip")

    # i think std overflows maybe at some point? 
    df["std"] = np.std(np.concatenate([np.fromstring(df["energies"].values[i][1:-1], sep=",", dtype=np.float32).reshape(1, -1) for i in range(df.shape[0])], axis=0)[:, -5:], axis=1)

    df = df[df["std"]<0.01] 

    df["pos"] = df["atom_positions"].apply(lambda x: np.fromstring(x[1:-1], sep=",", dtype=np.float64).reshape(-1).astype(np.float32)).values
    df["z"]   = df["atoms"].apply( lambda x: np.array([pt.symbol(a).number for a in x], dtype=np.uint8) )
    df["y"] = df["hlgap"]

    #df = df[["smile", "atoms", "z",  "homo", "lumo", "N", "std",  "pos", "y"]] # removed time 
    df = df[["z", "pos", "y"]] # just for traiing removed time 
    #df["std"]  = df["std"].to_numpy(np.float32) # 
    #df["homo"] = df["homo"].to_numpy(np.float32) # 
    #df["lumo"] = df["lumo"].to_numpy(np.float32) # 
    #df["N"]    = df["N"].to_numpy(np.uint8) # 

    df.to_parquet("/nethome/alexm/jaxdft-pre-experimental/qm1b_tmp/%s.parquet"%(filename.split("/")[-2]))
    return df.shape[0] 

for run in ["bigrun14", "bigrun15", "bigrun16"]:
    folders   = natsorted([a for a in os.listdir("data/generated/%s/"%run) if ".txt" not in a and "GDB" in a])
    print(len(folders), end="")
    folders   = [a for a in folders if os.path.exists("data/generated/%s/%s/data.csv"%(run, a))]
    print("->", len(folders))
    filenames = ["data/generated/%s/%s/data.csv"%(run, folders[i]) for i in range(len(folders))] 
    sizes     = [os.path.getsize("data/generated/%s/%s/data.csv"%(run, f)) for f in folders]
    indxs = np.argsort(sizes)
    filenames = np.array(filenames)[indxs].tolist()

    result_ = pqdm(filenames, square, n_jobs=100)
    print(np.sum([a for a in result_ if type(a) == type(2)]), len(result_))
    #print(">>>", np.max([a for a in result_ if type(a) == type(2)]))
    #square(filenames[-1])

320-> 298


QUEUEING TASKS | :   0%|          | 0/298 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/298 [00:00<?, ?it/s]

/nethome/alexm/miniconda3/envs/dft/lib/python3.8/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/nethome/alexm/miniconda3/envs/dft/lib/python3.8/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/nethome/alexm/miniconda3/envs/dft/lib/python3.8/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/nethome/alexm/miniconda3/envs/dft/lib/python3.8/site-packages/numpy/core/_methods.py:250: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/nethome/alexm/miniconda3/envs/dft/lib/python3.8/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


COLLECTING RESULTS | :   0%|          | 0/298 [00:00<?, ?it/s]

574293279 298
224-> 153


QUEUEING TASKS | :   0%|          | 0/153 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/153 [00:00<?, ?it/s]

In [3]:
import pyarrow.parquet as pq
import os 
from natsort import natsorted 

#files    = natsorted(os.listdir("qm1b_tmp/"))
files    = natsorted(os.listdir("/nethome/alexm/jaxdft-pre-experimental/qm1b_tmp/"))
parquets = natsorted(["qm1b_tmp/"+a for a in files if "parquet" in a])
df = pq.read_table(parquets)
df.shape

(97925, 3)

In [1]:
import pandas as pd 
pd.read_parquet("../petrignn/datasets/gdb100m/test/test.parquet")

,z,pos,y
0,"[9, 6, 6, 6, 8, 6, 7, 7, 7, 6, 1, 1, 1, 1]","[-4.5220003, -2.2504573, 2.3169382, -3.9694283...",6.649870
1,"[9, 6, 6, 6, 8, 6, 7, 7, 7, 6, 1, 1, 1, 1]","[-4.692202, -1.1123401, 2.8265004, -4.0558414,...",6.647062
2,"[9, 6, 6, 6, 8, 6, 7, 7, 7, 6, 1, 1, 1, 1]","[-1.8966775, -0.5760016, -5.180652, 0.51329523...",6.625340
3,"[9, 6, 6, 6, 8, 6, 7, 7, 7, 6, 1, 1, 1, 1]","[-4.4585404, -1.4275585, 3.3873675, -4.044052,...",6.739489
4,"[9, 6, 6, 6, 8, 6, 7, 7, 7, 6, 1, 1, 1, 1]","[-4.793129, -1.2303548, -2.5873125, -3.9083073...",6.757394
...,...,...,...
5804644,"[7, 7, 6, 8, 6, 6, 6, 8, 6, 8, 1, 1, 1, 1, 1, ...","[5.459736, 0.88411736, 0.34229085, 2.9631617, ...",5.298792
5804645,"[7, 7, 6, 8, 6, 6, 6, 8, 6, 8, 1, 1, 1, 1, 1, ...","[5.313457, 1.3426342, -1.4255189, 2.9915981, 0...",5.513345
5804646,"[7, 7, 6, 8, 6, 6, 6, 8, 6, 8, 1, 1, 1, 1, 1, ...","[5.454269, 1.1509848, -0.39161396, 3.0147963, ...",5.446056
5804647,"[7, 7, 6, 8, 6, 6, 6, 8, 6, 8, 1, 1, 1, 1, 1, ...","[5.3821034, 0.5583938, 0.51316136, 2.8419719, ...",5.535908


In [2]:
# can load all of it in 17min! (this is with f32, so might be able to halve?)
import os 
from tqdm.notebook import tqdm 
from natsort import natsorted 

files    = natsorted(os.listdir("qm1b_tmp/"))
parquets = natsorted([a for a in files if "parquet" in a])

dfs   = []

for parquet in tqdm(parquets):
  df   = pd.read_parquet("qm1b_tmp/" + parquet)
  dfs.append(df)


  0%|          | 0/565 [00:00<?, ?it/s]

In [2]:
from natsort import natsorted 
import os 
import polars as pl 

# roughly doubles the time 28s from pyarrow to 58s with this thing; + 14 s for the next thing. so this becomes 100 min loading instead of 4min

files    = natsorted(os.listdir("qm1b_tmp/"))
source_files = natsorted(["qm1b_tmp/"+a for a in files if "parquet" in a])[:10]

all_files = "\n".join(source_files)
print(f"\nLoading from files:\n{all_files}")

queries = [
  pl.scan_parquet(f).select("z", "pos", "y").cache()
  for f in source_files
]
df = pl.concat(queries).slice(offset=0, length=10**7).collect()

print(df)

#for chunk in df.iter_slices(n_rows=100000):
#  z, pos, y = zip(*chunk.to_numpy())



Loading from files:
qm1b_tmp/2_GDB10_f32True_grid0_backendipu_126945_128897.parquet
qm1b_tmp/3_GDB10_f32True_grid0_backendipu_134757_136709.parquet
qm1b_tmp/4_GDB10_f32True_grid0_backendipu_169911_171863.parquet
qm1b_tmp/5_GDB10_f32True_grid0_backendipu_142569_144521.parquet
qm1b_tmp/6_GDB10_f32True_grid0_backendipu_181629_183581.parquet
qm1b_tmp/7_GDB9_f32True_grid0_backendipu_15615_17349.parquet
qm1b_tmp/7_GDB10_f32True_grid0_backendipu_148428_150380.parquet
qm1b_tmp/7_GDB11_f32True_grid0_backendipu_181629_187487.parquet
qm1b_tmp/8_GDB10_f32True_grid0_backendipu_177723_179675.parquet
qm1b_tmp/8_GDB11_f32True_grid0_backendipu_111321_117179.parquet
shape: (10_000_000, 3)
┌─────────────┬───────────────────────────────────┬──────────┐
│ z           ┆ pos                               ┆ y        │
│ ---         ┆ ---                               ┆ ---      │
│ list[u8]    ┆ list[f32]                         ┆ f64      │
╞═════════════╪═══════════════════════════════════╪══════════╡
│ [8

In [4]:
for chunk in df.iter_slices(n_rows=100000):
  z, pos, y = zip(*chunk.to_numpy())

In [13]:
import pyarrow.parquet as pq
import os 
from natsort import natsorted 

files    = natsorted(os.listdir("qm1b_tmp/"))
parquets = natsorted(["qm1b_tmp/"+a for a in files if "parquet" in a])[:100]
df = pq.read_table(parquets)

In [14]:
df.shape

(148417609, 10)

In [5]:
import pyarrow as pa 
a = df
filtered_table = a.filter(pa.compute.less(a['std'], 0.01))

(17725172, 10)


In [ ]:
# can I get petrignn to just start training on the 1B, that'd be soo cool. 

In [7]:
pq.write_table(df, "qm1b/test.parquet")

In [8]:
pq.read_table("qm1b/test.parquet")

pyarrow.Table
smile: string
atoms: string
z: list<item: uint8>
  child 0, item: uint8
homo: float
lumo: float
N: uint8
std: float
time: float
pos: list<item: float>
  child 0, item: float
y: double
----
smile: [["On1cnn(NC=O)c1=N","On1cnn(NC=O)c1=N","On1cnn(NC=O)c1=N","On1cnn(NC=O)c1=N","On1cnn(NC=O)c1=N",...,"ON=C(C=O)C(C=O)C=O","ON=C(C=O)C(C=O)C=O","ON=C(C=O)C(C=O)C=O","ON=C(C=O)C(C=O)C=O","ON=C(C=O)C(C=O)C=O"],["ON=C(C=O)C(C=O)C=O","ON=C(C=O)C(C=O)C=O","ON=C(C=O)C(C=O)C=O","ON=C(C=O)C(C=O)C=O","ON=C(C=O)C(C=O)C=O",...,"O=C1NC2=C(C=CC2)C1=O","O=C1NC2=C(C=CC2)C1=O","O=C1NC2=C(C=CC2)C1=O","O=C1NC2=C(C=CC2)C1=O","O=C1NC2=C(C=CC2)C1=O"],...,["c1cc2=NN=CN=Cn2nn1","c1cc2=NN=CN=Cn2nn1","c1cc2=NN=CN=Cn2nn1","c1cc2=NN=CN=Cn2nn1","c1cc2=NN=CN=Cn2nn1",...,"On1nnc2C(=O)C3CN3c12","On1nnc2C(=O)C3CN3c12","On1nnc2C(=O)C3CN3c12","On1nnc2C(=O)C3CN3c12","On1nnc2C(=O)C3CN3c12"],["On1nnc2C(=O)C3CN3c12","On1nnc2C(=O)C3CN3c12","On1nnc2C(=O)C3CN3c12","On1nnc2C(=O)C3CN3c12","On1nnc2C(=O)C3CN3c12",...,"N=c1n(

In [1]:
import pyarrow.parquet as pq 
pq.read_table("qm1b/pyarrow.parquet")
pq

OSError: List index overflow.

: 

In [1]:
import pandas as pd 
df = pd.read_parquet("qm1b/pyarrow.parquet")

OSError: List index overflow.

In [6]:
import numpy as np 
from tqdm.notebook import tqdm 
tqdm.pandas()
df["pos"] = df["pos"].progress_apply(lambda x: x.astype(np.float16))

  0%|          | 0/3900598 [00:00<?, ?it/s]

In [7]:
df.to_parquet("test2.parquet")

ArrowNotImplementedError: Unhandled type for Arrow to Parquet schema conversion: halffloat

In [ ]:
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)
df.to_feather("qm1b/df.feather")

In [16]:
pd.read_feather("qm1b/df.feather")

,smile,atoms,z,homo,lumo,N,std,time,pos,y
0,On1cnn(NC=O)c1=N,ONCNNNCOCNHHHHH,"[8, 7, 6, 7, 7, 7, 6, 8, 6, 7, 1, 1, 1, 1, 1]",-2.345311,2.608682,55,0.000873,376101.781250,"[5.827063, 1.0317355, -0.7083503, 3.4275153, 0...",4.953993
1,On1cnn(NC=O)c1=N,ONCNNNCOCNHHHHH,"[8, 7, 6, 7, 7, 7, 6, 8, 6, 7, 1, 1, 1, 1, 1]",-2.275546,2.761140,55,0.001155,84.200005,"[-5.604111, 1.1757765, 0.05778461, -3.1909149,...",5.036686
2,On1cnn(NC=O)c1=N,ONCNNNCOCNHHHHH,"[8, 7, 6, 7, 7, 7, 6, 8, 6, 7, 1, 1, 1, 1, 1]",-2.391332,2.521584,55,0.001070,133.600006,"[-5.9132495, -0.38505128, -0.61481285, -3.4400...",4.912916
3,On1cnn(NC=O)c1=N,ONCNNNCOCNHHHHH,"[8, 7, 6, 7, 7, 7, 6, 8, 6, 7, 1, 1, 1, 1, 1]",-2.370459,2.592120,55,0.001070,81.100006,"[-5.6897364, 0.33897024, 0.35099038, -3.194107...",4.962579
4,On1cnn(NC=O)c1=N,ONCNNNCOCNHHHHH,"[8, 7, 6, 7, 7, 7, 6, 8, 6, 7, 1, 1, 1, 1, 1]",-2.436978,2.526828,55,0.000618,82.300003,"[-5.878585, -0.6040826, -0.18148863, -3.353134...",4.963806
...,...,...,...,...,...,...,...,...,...,...
667995,O=C1NCC2=NCCN2N1,OCNCCNCCNNHHHHHHHH,"[8, 6, 7, 6, 6, 7, 6, 6, 7, 7, 1, 1, 1, 1, 1, ...",-1.908326,3.069836,58,0.000618,104.999992,"[-5.38336, 3.5300062, 1.0764549, -3.6956263, 1...",4.978162
667996,O=C1NCC2=NCCN2N1,OCNCCNCCNNHHHHHHHH,"[8, 6, 7, 6, 6, 7, 6, 6, 7, 7, 1, 1, 1, 1, 1, ...",-2.142122,3.228512,58,0.001448,106.599998,"[-5.5638585, 3.6602335, -0.3452505, -3.8052273...",5.370634
667997,O=C1NCC2=NCCN2N1,OCNCCNCCNNHHHHHHHH,"[8, 6, 7, 6, 6, 7, 6, 6, 7, 7, 1, 1, 1, 1, 1, ...",-2.123112,3.230922,58,0.000756,107.900002,"[-5.4837356, 3.688127, 0.17725879, -3.7000666,...",5.354034
667998,O=C1NCC2=NCCN2N1,OCNCCNCCNNHHHHHHHH,"[8, 6, 7, 6, 6, 7, 6, 6, 7, 7, 1, 1, 1, 1, 1, ...",-2.127807,3.228734,58,0.000437,109.299988,"[5.790738, 2.7862897, 1.5203241, 3.8561738, 1....",5.356541


In [4]:
# i think the problem is all the atom positions stored as list, this takes forever to save! 
df = pd.concat(dfs) # 135m to 2G by adding atom postions
df.reset_index(drop=True, inplace=True)
df.to_parquet("qm1b/df.parquet")
#np.savez("qm1b/locs.npz", locs=locs)
#import pickle 
#pickle.dump(locs, open("locs.pickle", "wb"))
!ls -lah qm1b/

total 4.6G
drwxr-xr-x  2 alexm all    7 May 27 12:47 .
drwxr-xr-x 35 alexm all  459 May 27 12:16 ..
-rw-r--r--  1 alexm all    0 May 27 12:47 df.csv
-rw-r--r--  1 alexm all 1.5G May 27 12:37 df.feather
-rw-r--r--  1 alexm all 1.9G May 27 12:30 df.hdf
-rw-r--r--  1 alexm all 143M May 27 12:53 df.parquet
-rw-r--r--  1 alexm all 201M May 27 12:34 df.pickle
-rw-r--r--  1 alexm all 873M May 27 12:39 df_f32.feather
-rw-r--r--  1 alexm all   22 May 27 12:19 locs.npz


In [18]:
df.to_hdf("qm1b/df.hdf", key="a") # slower than parquet?

/tmp/ipykernel_92226/1114240224.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['smile', 'atoms', 'z', 'locs'], dtype='object')]

  df.to_hdf("qm1b/df.hdf", key="a")


TypeError: rename() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
df.to_pickle("qm1b/df.pickle")

In [9]:
df["locs"].values[0]

array([ 5.827063  ,  1.0317355 , -0.7083503 ,  3.4275153 ,  0.107191  ,
       -0.2731046 ,  2.8461204 , -2.1994011 ,  0.6168467 ,  0.31899458,
       -2.3834453 ,  0.7777409 , -0.75471944, -0.2444502 ,  0.01089214,
       -3.348137  ,  0.30252066, -0.10111679, -5.144159  , -1.5364552 ,
        0.65670764, -4.362598  , -3.63998   ,  1.4206738 ,  1.1881183 ,
        1.2599131 , -0.6261902 ,  0.98398364,  3.5514555 , -1.48208   ,
        6.7284646 ,  2.2162373 ,  0.4406668 ,  4.123091  , -3.7013693 ,
        1.1363839 , -3.918087  ,  2.0556333 , -0.7408562 , -7.1588707 ,
       -1.2443728 ,  0.61976993, -0.7567799 ,  4.4247875 , -1.7479838 ],
      dtype=float32)

In [10]:
d = pd.read_parquet("../petrignn/datasets/big_gdb8_sto3g/test/test.parquet")

In [11]:
d

,z,pos,y
0,"[6, 8, 6, 6, 6, 7, 7, 7, 1, 1, 1, 1, 1]","[-4.495205, 0.23872827, 0.2655194, -2.2186477,...",4.908727
1,"[7, 7, 6, 8, 7, 7, 6, 8, 1, 1, 1, 1, 1, 1]","[5.031974, 1.0477633, -0.78691244, 2.5052114, ...",5.594835
2,"[6, 6, 6, 6, 8, 6, 6, 6, 1, 1, 1, 1, 1, 1]","[5.133087, 3.0571103, 1.31773, 3.1669762, 1.97...",5.797307
3,"[6, 6, 6, 7, 7, 6, 6, 8, 1, 1, 1, 1, 1, 1]","[-3.1639757, 0.444887, 0.63958794, -0.61085707...",4.975348
4,"[6, 6, 6, 6, 7, 7, 6, 7, 1, 1, 1, 1, 1, 1, 1]","[3.4184594, -1.7734946, 0.5668929, 0.92932785,...",4.591497
...,...,...,...
1426525,"[8, 6, 8, 7, 6, 6, 6, 7, 1, 1]","[1.2840662, -0.70706034, -4.2914615, 0.0923704...",4.966056
1426526,"[6, 6, 7, 6, 8, 8, 7, 6, 1, 1, 1, 1, 1, 1]","[-2.8832853, -1.1241183, -0.08936391, -0.50957...",5.751349
1426527,"[6, 6, 6, 6, 6, 7, 6, 7, 1, 1, 1, 1, 1, 1]","[-4.2444363, -1.4364237, 0.86895883, -1.959656...",7.486381
1426528,"[6, 6, 6, 7, 6, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1]","[5.4422665, -1.3197862, 1.2723867, 3.9824693, ...",8.159339


In [8]:
import time  # perhaps storing locs as list of np array is ok? saves memory and may allous us to do d["locs"] = l
t0 = time.time()
d = pd.read_parquet("qm1b/df.parquet")
d["locs"] = np.load("qm1b/locs.npz")["locs"]
print(time.time()-t0)

51.93097901344299


In [ ]:
l = l.tolist()

In [11]:
indxs = np.random.permutation(df.shape[0])

In [13]:
d = d[indxs.tolist()]
l = l[indxs.tolist()]

KeyError: "None of [Index([65941161,  8322536, 45543450, 39443841, 62979457, 59037705,  7983726,\n       19360457, 46646622, 55121462,\n       ...\n       14595080, 23193646,  3007305, 31025584, 61784111, 11459249, 61700027,\n       49940572,  2878796, 39746859],\n      dtype='int64', length=66602000)] are in the [columns]"

In [10]:
# test 
# try to reproduce hlgap using pyscf andfix off-by-one! 
import os
os.environ["OMP_NUM_THREADS"] = "15"

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from tqdm import tqdm 
import scipy 
import os
import pyscf 
from pyscf import gto, scf
from pyscf import __config__ 
import argparse
from natsort import natsorted
import sys 
from rdkit import Chem  
from rdkit.Chem import AllChem
import csv 
import numpy as np 
import pyscf.dft
from pyscf.scf import hf
from pyscf.gto.mole import Mole
from pyscf import scf 
from pyscf import gto 
import time 
import pandas as pd 
import re 
#from pyscf_utils.fast_grid import build_grid
from rdkit import Chem # 
from rdkit.Chem import AllChem
from rdkit import RDLogger

__config__.dft_rks_RKS_grids_level = 0 #args.plevel 

angstrom_to_bohr = 1.88973
hartree_to_eV    = 27.2114

def get_atom_string(atoms, locs):
    atom_string = atoms 
    atoms = re.findall('[a-zA-Z][^A-Z]*', atoms)
    str = ""
    #print(locs)
    for atom, loc in zip(atoms, locs): 
      str += "%s %4f %4f %4f; "%((atom,) + tuple(loc) )
    return atom_string, str 


def recompute(atoms, locs):
  locs = locs[locs!=0].reshape(-1, 3)
  #print(locs)
  atom_string, string = get_atom_string(" ".join(atoms), locs)

  #mol = Mole() 
  #mol = build_mol(mol, atom=string, unit="Bohr", basis="sto3g", spin=0, verbose=0)
  mol = Mole( atom=string, unit="Bohr", basis="sto3g", spin=0, verbose=0)
  mol.build()

  n_electrons      = mol.nelectron                                                     
  n_electrons_half = n_electrons//2 

  mf = scf.RKS(mol) 
  mf.xc = "b3lyp" 

  mf.diis_space = 9 
  lumo = n_electrons_half
  homo = lumo - 1

  pyscf_energy    = mf.kernel()  

  pyscf_homo = mf.mo_energy[homo]*hartree_to_eV
  pyscf_lumo = mf.mo_energy[lumo]*hartree_to_eV
  pyscf_hlgap = np.abs(pyscf_homo - pyscf_lumo )
  #print(df["hlgap"][:50])
  return pyscf_hlgap

  # so the first atom positions and smile string wasn't saved 
  # instead, we saved the second smile string and second atom locations, and theese where saved together with the result of the first computation! 
  # => if we try to recompute pyscf on (atomstring[0], locs[0]) we get hlgap[1]
  # this means we can recover the dataset! we just have to, for each smile string, offset it by one!

#atoms = df["atoms"][0:1]
#locs = alllocs[0] 

for j in range(100):
  
  print(j, recompute(df["atoms"].values[j], locs[j]), df["hlgap"].values[j])

0 4.953149244910048 4.953993313487519
1 5.037165005155293 5.036685702192981


KeyboardInterrupt: 